## Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

* Create a Jupyter Notebook called `database_engineering.ipynb` and use this to complete all of your Database Engineering work.

* Use Pandas to read your cleaned measurements and stations CSV data.

* Use the `engine` and connection string to create a database called `hawaii.sqlite`.

* Use `declarative_base` and create ORM classes for each table.

  * You will need a class for `Measurement` and for `Station`.

  * Make sure to define your primary keys.

* Once you have your ORM classes defined, create the tables in the database using `create_all`.


In [1]:
#Dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
# Use the `engine` and connection string to create a database called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [4]:
#Use `declarative_base` and create ORM classes for each table.
Base = declarative_base()

In [5]:
#Create Measurement Class.
class Measurement(Base):
    __tablename__ = "measurements"
    id = Column(Integer, primary_key=True )
    station = Column(String(11))
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
                return f"id={self.id}, name={self.name}"
    

In [6]:
#Create Station class
class Station(Base):
    __tablename__ = "stations"
    id = Column(Integer, primary_key=True )
    station = Column(String(11))
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [7]:
#Once you have your ORM classes defined, create the tables in the database using `create_all`.
Base.metadata.create_all(engine)

In [9]:
# Load the cleaned csv files into a pandas dataframe
measurements_df = pd.read_csv("Resources/clean_mesurements.csv")
stations_df = pd.read_csv("Resources/clean_stations.csv")


In [10]:
#Use Orient='records' to create a list of data to write
measurements_data = measurements_df.to_dict(orient='records')
stations_data = stations_df.to_dict(orient='records')
measurements_data[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [11]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [12]:
# Save the reference to the `mesurements` and `stations' table as a variable called `table`
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [13]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [14]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurements_table.insert(), measurements_data)
conn.execute(stations_table.insert(), stations_data)

In [15]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [16]:
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]